 [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)


## Imports and Configuration
We'll Start by importing the packages we use and setting some notebook defaults. 

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm.notebook import tqdm
import warnings
import random

from scipy import stats     # norm, skew, kurtosis, boxcox
from scipy import special  # boxcox1p, inv_boxcox, inv_boxcox1p

### sklearn 
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso, LogisticRegression,RidgeCV, LassoLars, BayesianRidge, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split, KFold, cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, PowerTransformer, StandardScaler, PolynomialFeatures, scale
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# xgboost
from xgboost import XGBRegressor

In [2]:
random.seed(42)
np.random.seed(42)

tqdm().pandas()
pd.set_option('display.max_colwidth', None)
pd.set_option('use_inf_as_na', True)
sns.set_style( 'whitegrid' )

# Set Matplotlib defaults
#plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')

#Limiting floats output to 3 decimal points
pd.set_option(
    'display.float_format',
    lambda x: '{:.3f}'.format(x)
) 

## Set up code checking
os.symlink() method is used to create symbolic link. This method creates symbolic link pointing to source named destination.

In [3]:
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 

### Fast Run!

In [4]:
"""
# Remove rows with missing target, separate target from predictors
df_train = pd.read_csv("../input/train.csv", index_col=0)
df_test = pd.read_csv("../input/test.csv", index_col=0)

x = df_train.dropna(axis=0, subset=['SalePrice'])
y = df_train.SalePrice
x = df_train.drop(['SalePrice'], axis=1)

# Select numerical columns
categorical_features = x.select_dtypes(include=np.object).columns.tolist()
numerical_features = x.select_dtypes(include='number').columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', GradientBoostingRegressor(n_estimators=200, learning_rate=0.09))
])

pipeline.fit(x, y)

Save_Submit(pipeline, df_test)
""";

#### Let's read our train dataset.

In [5]:
# Read the data
df = pd.read_csv(
    "../input/train.csv", 
    index_col=0
)
#df.reset_index(drop=True, inplace=True)
df

In [6]:
data_description = {
    'SalePrice': 'sale price in dollars (our target)',
    'MSSubClass': 'The building class',
    'MSZoning': 'The general zoning classification',
    'LotFrontage': 'Linear feet of street connected to property',
    'LotArea': 'Lot size in square feet',
    'Street': 'Type of road access',
    'Alley': 'Type of alley access',
    'LotShape': 'General shape of property',
    'LandContour': 'Flatness of the property',
    'Utilities': 'Type of utilities available',
    'LotConfig': 'Lot configuration',
    'LandSlope': 'Slope of property',
    'Neighborhood': 'Physical locations within Ames city limits',
    'Condition1': 'Proximity to main road or railroad',
    'Condition2': 'Proximity to main road or railroad (if a second is present)',
    'BldgType': 'Type of dwelling',
    'HouseStyle': 'Style of dwelling',
    'OverallQual': 'Overall material and finish quality',
    'OverallCond': 'Overall condition rating',
    'YearBuilt': 'Original construction date',
    'YearRemodAdd': 'Remodel date',
    'RoofStyle': 'Type of roof',
    'RoofMatl': 'Roof material',
    'Exterior1st': 'Exterior covering on house',
    'Exterior2nd': 'Exterior covering on house (if more than one material)',
    'MasVnrType': 'Masonry veneer type',
    'MasVnrArea': 'Masonry veneer area in square feet',
    'ExterQual': 'Exterior material quality',
    'ExterCond': 'Present condition of the material on the exterior',
    'Foundation': 'Type of foundation',
    'BsmtQual': 'Height of the basement',
    'BsmtCond': 'General condition of the basement',
    'BsmtExposure': 'Walkout or garden level basement walls',
    'BsmtFinType1': 'Quality of basement finished area',
    'BsmtFinSF1': 'Type 1 finished square feet',
    'BsmtFinType2': 'Quality of second finished area (if present)',
    'BsmtFinSF2': 'Type 2 finished square feet',
    'BsmtUnfSF': 'Unfinished square feet of basement area',
    'TotalBsmtSF': 'Total square feet of basement area',
    'Heating': 'Type of heating',
    'HeatingQC': 'Heating quality and condition',
    'CentralAir': 'Central air conditioning',
    'Electrical': 'Electrical system',
    '1stFlrSF': 'First Floor square feet',
    '2ndFlrSF': 'Second floor square feet',
    'LowQualFinSF': 'Low quality finished square feet (all floors)',
    'GrLivArea': 'Above grade (ground) living area square feet',
    'BsmtFullBath': 'Basement full bathrooms',
    'BsmtHalfBath': 'Basement half bathrooms',
    'FullBath': 'Full bathrooms above grade',
    'HalfBath': 'Half baths above grade',
    'Bedroom': 'Number of bedrooms above basement level',
    'Kitchen': 'Number of kitchens',
    'KitchenQual': 'Kitchen quality',
    'TotRmsAbvGrd': 'Total rooms above grade (does not include bathrooms)',
    'Functional': 'Home functionality rating',
    'Fireplaces': 'Number of fireplaces',
    'FireplaceQu': 'Fireplace quality',
    'GarageType': 'Garage location',
    'GarageYrBlt': 'Year garage was built',
    'GarageFinish': 'Interior finish of the garage',
    'GarageCars': 'Size of garage in car capacity',
    'GarageArea': 'Size of garage in square feet',
    'GarageQual': 'Garage quality',
    'GarageCond': 'Garage condition',
    'PavedDrive': 'Paved driveway',
    'WoodDeckSF': 'Wood deck area in square feet',
    'OpenPorchSF': 'Open porch area in square feet',
    'EnclosedPorch': 'Enclosed porch area in square feet',
    '3SsnPorch': 'Three season porch area in square feet',
    'ScreenPorch': 'Screen porch area in square feet',
    'PoolArea': 'Pool area in square feet',
    'PoolQC': 'Pool quality',
    'Fence': 'Fence quality',
    'MiscFeature': 'Miscellaneous feature not covered in other categories',
    'MiscVal': '$Value of miscellaneous feature',
    'MoSold': 'Month Sold',
    'YrSold': 'Year Sold',
    'SaleType': 'Type of sale',
    'SaleCondition': 'Condition of sale',
}

In [7]:
def explore_target( y = df['SalePrice'] ):
    skew_y = stats.skew( y )
    # kurtosis a measure of the combined weight 
    # of a distribution's tails relative to the center of the distribution.
    kurtosis_y = stats.kurtosis( y )
    mu, sigma = stats.norm.fit( y )
    print( f"y: mu = {mu:.2f} and sigma = {sigma:.2f}, skew = {skew_y:.2f} kurtosis = {kurtosis_y:.2f}")
    #------------
    #log_y = np.log( y )
    log_y = np.log1p( y )
    log_skew_y = stats.skew( log_y )
    log_kurtosis_y = stats.kurtosis( log_y )
    log_mu, log_sigma = stats.norm.fit( log_y )
    print( f"Log y: mu = {log_mu:.2f} and sigma = {log_sigma:.2f}, skew = {log_skew_y:.2f} kurtosis = {log_kurtosis_y:.2f}")
    #------------
    # A Box Cox transformation is a transformation of non-normal dependent variables 
    # into a normal shape.
    #boxcox_y = stats.boxcox( y )[0]
    # try different alpha values "optimized value"  between 0 and 1
    boxcox_y = special.boxcox1p(y, 0.35)
    boxcox_skew_y = stats.skew( boxcox_y )
    boxcox_kurtosis_y = stats.kurtosis( boxcox_y )
    boxcox_mu, boxcox_sigma = stats.norm.fit( boxcox_y )
    print( f"boxcox y: mu = {boxcox_mu:.2f} and sigma = {boxcox_sigma:.2f}, skew = {boxcox_skew_y:.2f} kurtosis = {boxcox_kurtosis_y:.2f}")

    # graph
    fig, axes = plt.subplots( 3, 2, figsize = ( 14, 10 ) );
    
    sns.distplot(y, fit=stats.norm, ax=axes[0,0])
    axes[0,0].set_xlabel('SalePrice distribution');
    axes[0,0].set_title( 'Probplot against normal distribution' );
    axes[0,0].legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')

    prob1 = stats.probplot( y, plot = axes[0,1] );
    axes[0,1].set_xlabel('');
    axes[0,1].set_title( 'probability plot' );
    #-----------------------------------------------------
    sns.distplot(log_y, fit=stats.norm, ax=axes[1,0])
    axes[1,0].set_xlabel('Log SalePrice distribution');
    axes[1,0].set_title( 'Probplot against normal distribution' );
    axes[1,0].legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(log_mu, log_sigma)], loc='best')

    prob2 = stats.probplot( log_y, plot = axes[1,1] );
    axes[1,1].set_title( 'probability plot after log transform' );
    #-----------------------------------------------------
    sns.distplot(boxcox_y, fit=stats.norm, ax=axes[2,0])
    axes[2,0].set_xlabel('boxcox SalePrice distribution');
    axes[2,0].set_title( 'Probplot against normal distribution' );
    axes[2,0].legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(boxcox_mu, boxcox_sigma)], loc='best')

    prob3 = stats.probplot( boxcox_y, plot = axes[2,1] );
    axes[2,1].set_title( 'probability plot after boxcox transform' );

explore_target(df['SalePrice'])

In [8]:
def explore_feature(x, y=df['SalePrice']):
    #log_y = np.log(y)
    log_y = np.log1p(y)
    
    # regression
    m, b = np.polyfit(x, log_y, 1) # m = slope, b = intercept
    reg = m*x+b
     
    # let's try polynomial fit  "linear" 
    curve_fit = np.polyfit(x, log_y, 1)
    print(f"curve fit 2: {curve_fit}")
    y_poly_1 = np.exp(curve_fit[1]) * np.exp(curve_fit[0]*x)
    
    # let's try polynomial fit l"inear with log y"
    curve_fit_gla = np.polyfit(x, y, 2)
    print(f"curve fit 2: {curve_fit_gla}")
    y_poly_2 = curve_fit_gla[2] + curve_fit_gla[1]*x + curve_fit_gla[0]*x**2
    
    # graph
    fig, axes = plt.subplots( 3, 3, figsize = ( 14, 10 ) );
    # -----------------------------------------------------------------
    sns.regplot(x=x, y=y, marker="+", ax=axes[0,0] )
    axes[0,0].set_ylabel(y.name, fontsize=13)
    axes[0,0].set_xlabel(x.name, fontsize=13)
    axes[0,0].set_title( 'Regression Line' );
    
    sns.regplot(x=x, y=log_y, marker="+", ax=axes[0,1] )
    axes[0,1].set_ylabel(y.name, fontsize=13)
    axes[0,1].set_xlabel(x.name, fontsize=13)
    axes[0,1].set_title( 'Regression Line  with log y' );
    
    axes[0,2].scatter(x, y_poly_1, c='red')
    axes[0,2].plot(x, y, "+")
    axes[0,2].set_ylabel(y.name, fontsize=13)
    axes[0,2].set_xlabel(x.name, fontsize=13)
    axes[0,2].set_title( "polynomial fit  'linear' " );
    # -----------------------------------------------------------------
    axes[1,0].scatter(x, np.log(y_poly_1), c='red')
    axes[1,0].plot(x, log_y, "+")
    axes[1,0].set_ylabel(f"Log {y.name}", fontsize=13)
    axes[1,0].set_xlabel(x.name, fontsize=13)
    axes[1,0].set_title( "polynomial fit  'linear' with log y" );

    axes[1,1].scatter(x, y_poly_2, c='red')
    axes[1,1].plot(x, y, "+")
    axes[1,1].set_ylabel(y.name, fontsize=13)
    axes[1,1].set_xlabel(x.name, fontsize=13)
    axes[1,1].set_title( "polynomial fit " );
    
    axes[1,2].scatter(x, np.log(y_poly_2), c='red')
    axes[1,2].plot(x, log_y, "+")
    axes[1,2].set_ylabel(f"Log {y.name}", fontsize=13)
    axes[1,2].set_xlabel(x.name, fontsize=13)
    axes[1,2].set_title( "polynomial fit  with log y" );
    # -----------------------------------------------------------------
    sns.boxplot(x=x, y=y, ax=axes[2,0] )
    axes[2,0].set_xlabel( f"{x.name}" );
    axes[2,0].set_title( 'box plot' );
    
    sns.boxplot(x=x, y=log_y, ax=axes[2,1] )
    axes[2,1].set_xlabel( f"{x.name}" );
    axes[2,1].set_title( 'box plot' );
    
    sns.histplot(x=x, kde=True, ax=axes[2,2] )
    axes[2,2].set_xlabel( f"{x.name}" );
    axes[2,2].set_title( 'hist plot' );
    
explore_feature(df.GrLivArea, df.SalePrice)

Let's create def r_squared function

In [9]:
def r_squared(x, y):
    return np.corrcoef(x, y)[0,1]**2

In [10]:
def gla_feature(x, y):
    def add_gla(row, x=x, y=y):
        p = np.polyfit(x, y, 2)
        return (p[2] + p[1]*row.GrLivArea + p[0]*(row.GrLivArea**2))
    return df.apply(lambda row: add_gla(row), axis=1)

#r_squared( df.GrLivArea, df.SalePrice), r_squared( a, df.SalePrice), 

###  value that shows up only once in the dataset
delete values since the it is guaranteed to create a std of zero. DON'T apply this function in test dataset.

In [11]:
def only_once_values(df):
    X = df.copy()
    n_rows = X.shape[0]
    only_once = {}
    for name in X.select_dtypes(exclude=['number']):
        once = X[name].value_counts() <= 1
        if (once).any():
            #print(f"\n========={name}=============")
            #print(X[name].value_counts())
            # store items that that shows up only once
            only_once[name] = once[once].index.tolist()
            for i in once[once].index.tolist():
                X = X[X[name] != i]
    #print(only_once)
    print(f"{n_rows-X.shape[0]} rows deleted :)")
    return X
        
#df = only_once_values(df)

###  couple of issues here
We need to replace "Brk Comm" items to "BrkComm" and find items that it's GarageYrBlt is earier than YearBuilt.

In [12]:
def Clean(df):
    # replace "Brk Comm" items to "BrkComm"
    df["Exterior2nd"] = df["Exterior2nd"].replace({
        "Brk Cmn": "BrkComm"
    })
    # find items that it's GarageYrBlt is earier than YearBuilt
    #print(df[df.apply(lambda x: x['GarageYrBlt'] < x['YearBuilt'], axis = 1)][['GarageYrBlt', 'YearBuilt']] )
    # replace corrupted items with the year the house was built
    df["GarageYrBlt"] = df["GarageYrBlt"].where(df.GarageYrBlt > 2010, df.YearBuilt)
    return df

df = Clean(df)

### Scale
Scale generally means to change the range of the values. The shape of the distribution doesn’t change. Think about how a scale model of a building has the same proportions as the original, just smaller. That’s why we say it is drawn to scale. The range is often set at 0 to 1.
### Standardize 
Standardize generally means changing the values so that the distribution’s standard deviation equals one. Scaling is often implied.
### Normalize 
Normalize can be used to mean either of the above things (and more!). I suggest you avoid the term normalize, because it has many definitions and is prone to creating confusion.

In [13]:
def Standalizer(X):
    return (X - X.mean(axis=0)) / X.std(axis=0)

def Numericalscaler(X):
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

Let's check for duplicated rows in our dataset.

In [14]:
df.duplicated().any() 

### Missing Value?
Theoretically, 25 to 30% is the maximum missing values are allowed, beyond which we might want to drop the variable from analysis.

In [15]:
def Missing_Percentage(df, drop_percentage=25):
    X = df.copy()
    missing_values = X.isnull().sum()
    missing_values = missing_values[missing_values > 0].sort_values(ascending = False)
    NAN_col = list(missing_values.to_dict().keys())
    missing_values_data = pd.DataFrame(missing_values)
    missing_values_data.reset_index(level=0, inplace=True)
    missing_values_data.columns = ['Feature','Number of Missing Values']
    missing_values_data['NA Percentage'] = (100.0 * missing_values_data['Number of Missing Values']) / X.shape[0]
    missing_values_data['Describe'] = missing_values_data.Feature.map(data_description)
    missing_values_data['dtype'] = [ X[name].dtype for name in missing_values_data.Feature ]
    
    corr_matrix = X.corr()
    corr_dict = {}
    for i in NAN_col:
        if i in corr_matrix.index:
            corr_dict[i] = np.abs(corr_matrix[i]).drop([i]).sort_values(ascending=False).index[0]
        else:
            corr_dict[i] = '-'
    missing_values_data['Correlated Feature'] = missing_values_data.Feature.map(corr_dict)
    # features to drop
    #drop_features = missing_values_data[missing_values_data['NA Percentage'] > drop_percentage].Feature.tolist()
    return missing_values_data.set_index(missing_values_data.Feature)

missing_df = Missing_Percentage(df, 25)
missing_df

In [16]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=missing_df.index, y=missing_df['NA Percentage']);
plt.xlabel('Features', fontsize=15);
plt.ylabel('Percent of missing values', fontsize=15);
plt.title('Percent missing data by feature', fontsize=15);

### Handle Missing Value
Handling missing values now will make the feature engineering go more smoothly. As a start we will deal with features with null value discovered in the dataset individually.

* Pool quality feature (PoolQC)   
We would fill all missing values with 'None' since the NAN values here simply represents that the house does not have a pool.

* Miscellaneous (MiscFeature)     
It represents Miscellaneous feature not covered in other categories. As PoolQC feature we will fill all missing values with 'None'.

* Alley access (Alley)      
It represents Type of alley access so we will fill all missing values with 'None'.

* Fence quality (Fence)   
We will fill all missing values with 'None'.

* Fireplace quality (FireplaceQu)    
We would fill all missing values with most frequently used value.

* LotFrontage     
Here we would first fill all missing values by taking the mean of the LotFrontage values of all groups having the same values of 1stFlrSF. This is because LotFrontage has a high correlation with 1stFlrSF. However, there can be cases where all the LotFrontage values corresponding to a particular 1stFlrSF value can be missing. To tackle such cases we would then fill it by using interpolate function of pandas to fill missing values linearly.

* Garage (GarageType, GarageFinish, GarageQual, GarageCond)   
No garage? so we will fill all missing values with 'None'.

* Basement (BsmtExposure, BsmtFinType2, BsmtFinType1, BsmtCond,BsmtQual)  
No basement? so we will fill all missing values with 'None'.

* Masonry veneer (MasVnrArea, MasVnrType)  
It represents Type of Masonry veneer so we will fill all missing values with 'None' forMasVnrType feature and 0 for MasVnrArea feature.

* Electrical system (Electrical)    
We would fill all missing values with most frequently used value.

Otherwise, for categorical features we will use most frequently and foe numerical features

##### Other ideas:
    - replace infinity values with null values.
    - remove rows with missing target, separate target from predictors.

In [17]:
def ImputeData(df, name):
    from sklearn.impute import KNNImputer
    
    numerical_df = df.select_dtypes(include='number')
    imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
    imputer.fit(numerical_df)
    X_transformed = imputer.transform(numerical_df)
    df_miss = pd.DataFrame(
        X_transformed, 
        columns = numerical_df.columns
    )
    #all_data[col_to_impute] = df_miss[col_to_impute]
    return df_miss[name]

ImputeData(df, 'LotFrontage')

In [18]:
def Impute(df):
    X = df.copy()
    #print(f"Null Columns are { X.columns[ X.isnull().any() ].tolist()}")
    
    # Remove rows with missing target, separate target from predictors
    #print( df_train.SalePrice.isnull().sum() )
    #X.dropna(axis=0, subset=['SalePrice'], inplace=True)
    
    # Replace infinity values with null values
    #print( np.isfinite(X.all()).sum() )
    #X.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # MasVnrArea feature
    X['MasVnrArea'].fillna(0, inplace=True)
    
    # LotFrontage feature
    X['LotFrontage'].fillna(
        X.groupby('1stFlrSF')['LotFrontage'].transform('mean'), 
        inplace = True
    )
    X['LotFrontage'].interpolate(
        method = 'linear',
        inplace = True
    )

    for name in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'LotFrontage', 'GarageType',\
                 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtExposure', 'BsmtFinType2',\
                 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType']: 
        X[name].fillna('None', inplace=True)
    
    for name in X.select_dtypes(include=['number']).columns:
        # fill nan with mean 
        X[name].fillna(df.mean(axis=0)[name], inplace=True)
        
    for name in X.select_dtypes(include=np.object).columns:
        # including 'FireplaceQu', 'Electrical' features
        # fill nan with most frequency
        most_frequently = X[name].value_counts().index[0]
        X[name].fillna(most_frequently, inplace=True)
        
    # check if no null value left
    assert(X.isnull().sum().sum() == 0)
    #print( np.isfinite(X.all()).sum() )
    #assert(np.isfinite(X.all()).sum() == 0) 
     
    return X

df = Impute(df)

### Show categorical data description

In [19]:
df_desc = df.describe(include=np.object).T
df_desc['describe'] = df_desc.index.map(data_description)
df_desc['categories'] = pd.DataFrame({
        name: ', '.join(df[name].unique().tolist()) 
        for name in df.select_dtypes(exclude='number').columns.tolist()
    }, index=['categories']).T
df_desc

# Outliers Treatment

    - Flooring and Capping: quantile-based technique.
    - Trimming: remove and completely drop all the outliers.
    - Replacing outliers: with the mean, median, mode, or other values. 'adviced not to use mean values'
   

#### Remove all rows that have outliers in at least one column    

* For each column, it first computes the Z-score of each value in the column, relative to the column mean and standard deviation.
* It then takes the absolute Z-score because the direction does not matter, only if it is below the threshold.
* all(axis=1) ensures that for each row, all column satisfy the constraint.
    Finally, the result of this condition is used to index the dataframe.


In [20]:
from scipy import stats
from time import time

standard_deviations = 3

#outliers = df[ df.apply(lambda x: np.abs(x - x.mean()) / x.std() < standard_deviations).all(axis=1) ]
#outliers = df[ ( np.abs( stats.zscore( df ) ) < standard_deviations ).all(axis=1) ]
# drop outliers columns, first we add 
#pd.concat([df, outliers]).drop_duplicates(keep=False)

# Select and remove outliers from dtata
def remove_outliers(df, standard_deviations=3):
    X = df.copy()
    z_score = stats.zscore( X.select_dtypes(include='number') )
    return X[ ~(  np.abs(z_score)  < standard_deviations ).all(axis=1) ]

remove_outliers(df, standard_deviations=3)

Let's remove the bottom right two with extremely large GrLivArea that are of a low price. These values are huge oultliers. Therefore, we can safely delete them.

Think this deletion is safe to do as there are only 2 points and they seem to be abnormal, possibly data error.

In [21]:
df[(df['OverallQual']>9) & (df['SalePrice']<220000)]

In [22]:
df[(df['GrLivArea']>4000) & (df['SalePrice']<300000)]

In [23]:
def delete_uninformative(df):
    # extremely large areas for very low prices
    X = df.copy()
    X = X.drop(
        X[
            (X['OverallQual']>9) & (X['SalePrice']<220000)
        ].index
    )
    X = X.drop(
        X[
            (X['GrLivArea']>4000) & (X['SalePrice']<300000)
        ].index
    )
    return X

df = delete_uninformative(df)# dropped 2 rows from training data

In [24]:
# filtering on a column by Interquartile Range(IQR)
def Interquartile_Range(df):
    iqr = pd.DataFrame({
        'Q1': df.quantile(0.25), # first quantile,
        'Q3': df.quantile(0.75), # second quantile,
    })
    iqr['IQR'] = iqr.Q3 - iqr.Q1
    return iqr


def outlier_detect_new(df, name, iqr_df, whisker_width=1.5):
    """Remove outliers from a dataframe by column, including optional 
       whiskers, removing rows for which the column value are 
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    Args:
        df (`:obj:pd.DataFrame`): A pandas dataframe to subset
        column (str): Name of the column to calculate the subset from.
        whisker_width (float): Optional, loosen the IQR filter by a
                               factor of `whisker_width` * IQR.
    Returns:
        (`:obj:pd.DataFrame`): Filtered dataframe
    """
    lower_bound = iqr_df.Q1[name] - whisker_width * iqr_df.IQR[name]
    upper_bound = iqr_df.Q3[name] + whisker_width * iqr_df.IQR[name]
    return df[ 
        ((df[name] < lower_bound) | (df[name] > upper_bound)) 
    ]

def lower_outlier(df, name, iqr_df, whisker_width=1.5):
    lower_bound =  iqr_df.Q1[name] - whisker_width * iqr_df.IQR[name]
    return df[ ( df[name] < lower_bound ) ]

def upper_outlier(df, name, iqr_df, whisker_width=1.5):
    upper_bound =  iqr_df.Q3[name] + whisker_width * iqr_df.IQR[name]
    return df[ ( df[name] < upper_bound ) ]

In [25]:
# Remove rows with missing target, separate target from predictors
target = df.SalePrice
feaures = df.drop(['SalePrice'], axis=1)

### Encode the Statistical Data Type
Here we will use one hot encoder for the features that has high cardinality columns and the features with unordered categories.
otherwise we will use label encoder. maximum cardinality in traing dataset id 25 for "Neighborhood" column
* "Cardinality" means the number of unique values in a column


#### Label Encoder Methods

* pd.factorize()
* np.where(series == 'yes', 1, 0)
* pd.Categorical/astype('category'): series.astype('category').cat.codes
* series.replace({'yes' : 1, 'no' : 0})
* series.replace({r'^(?!yes).*$' : 0}, regex=True).astype(bool).astype(int)
* dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [26]:
def label_encode(df):
    X = df.copy()
    for name in X.select_dtypes(include=np.object):
        X[name], _ = X[name].factorize()
    return X

In [27]:
#  Avoid OneHot for high cardinality columns

#print( df.select_dtypes(include=np.object).nunique().max() )  # 25
    
def Encoder(df, max_number=15 ):
    X = df.copy()
    #print(X.select_dtypes(include=np.object).nunique())
    
    #for colname in X.select_dtypes(["category"]):
    #    X[colname] = X[colname].cat.codes
    
    for name in X.select_dtypes(include=np.object):
        if X[name].nunique() > max_number: # and ordered features
            X[name], _ = X[name].factorize()
            
        else:
            X = pd.get_dummies(X, prefix=[f"{name}_"], columns=[name])
            
    return X

df = Encoder(df, max_number=5)
Encoder(df, max_number=5)

##  Feature Utility Metric
Use a metric called "mutual information" to compute a utility score for a feature, giving you an indication of how much potential the feature has.
It's a lot like correlation in that it measures a relationship between two quantities. 
The advantage of mutual information is that it can detect any kind of relationship, while correlation only detects linear relationships.

In [28]:
# All discrete features should now have integer dtypes
discrete_features = feaures.dtypes == int

In [29]:
def make_mi_scores(X, y):
    # All discrete features should now have integer dtypes
    X = label_encode(X)
    #discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    discrete_features = (X.dtypes == int) 
    
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(feaures, target)

Here we create bar plot based on mutual information scores.

In [30]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    
plot_mi_scores(mi_scores[:10])

Now we have a number of features that are highly informative. the top scoring features will usually pay-off the most during feature development, so it could be a good idea to focus efforts on those. On the other hand, training on uninformative features can lead to overfitting. So, the features with 0.0 scores we'll drop entirely.

In [31]:
def drop_uninformative(df, mi_scores, threshold=0.0):
    return df.loc[:, mi_scores > threshold]

drop_uninformative(feaures, mi_scores, 0.0)

In [32]:
top_features = df.corr()[['SalePrice']].sort_values(by=['SalePrice'],ascending=False)[:20]

plt.figure(figsize=(5,10));
sns.heatmap(top_features, cmap='rainbow', annot=True, annot_kws={"size": 16}, vmin=-1);

# Creating Features
 Here we apply some strategies for creating features in Pandas.
 
 #### Creating features strategies
* Mathematical Transforms
* Counts
* Building-Up and Breaking-Down Features
* Group Transforms
* K-means
* PCA

In [33]:
df_desc = feaures.describe(include=['number']).T
df_desc['describe'] = df_desc.index.map(data_description)

df_desc.dropna() # drop new columns 

In [50]:
def mathematical_transforms(df):
    X = pd.DataFrame(
        index = df.index
    )  # dataframe to hold new features
    X["LivLotRatio"] = df.GrLivArea / df.LotArea  # create a single null value
    X["Spaciousness"] = (df['1stFlrSF'] + df['2ndFlrSF']) / df.TotRmsAbvGrd # create a single null value
    X["Feature1"] = df.GrLivArea + df.TotalBsmtSF
    X["Feature2"] = df.YearRemodAdd * df.TotalBsmtSF    
    #X['Total_Square_Feet'] = (df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF'])
    X['Total_Bath'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))
    X['Total_Porch_Area'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
    X['SqFtPerRoom'] = df['GrLivArea'] / (df['TotRmsAbvGrd'] + df['FullBath'] + df['HalfBath'] + df['KitchenAbvGr'])
    return X

#df.join(mathematical_transforms(df))#.isnull().sum().sum() # 2 null values
mathematical_transforms(df)

In [35]:
def interactions(df):
    X = pd.get_dummies(df.BldgType, prefix="Bldg") # deleted after using one hot
    X = X.mul(df.GrLivArea, axis=0)
    return X

#df.join(interactions(df))#.isnull().sum().sum()
#interactions(df)

In [36]:
def counts(df):
    X = pd.DataFrame(
         index=df.index
    )
    X["PorchTypes"] = df[[
        "WoodDeckSF",
        "OpenPorchSF",
        "EnclosedPorch",
        "3SsnPorch",
        "ScreenPorch",
    ]].gt(0.0).sum(axis=1)
    return X

#df.join(counts(df)).isnull().sum().sum()#
counts(df)

In [37]:
def Building_Up_Breaking_Down(df, col):
    X = pd.DataFrame(
        df.index
    )
    #X[f"new{col}"] = df[col].str.split("_", n=1, expand=True)[0]
    return X

#df.join(Building_Up_Breaking_Down(df, col=''))#.isnull().sum().sum()
#Building_Up_Breaking_Down(df, col='')

In [38]:
def group_transforms(df):
    X = pd.DataFrame()
    X["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")
    return X

#df.join(group_transforms(df))#.isnull().sum().sum()
group_transforms(df)

#### KMeans
An nsupervised algorithm we used to create features was k-means clustering. We saw that you could either use the cluster labels as a feature (a column with 0, 1, 2, ...) or you could use the distance of the observations to each cluster. We saw how these features can sometimes be effective at untangling complicated spatial relationships.

In [39]:
cluster_features = [
    "LotArea",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "GrLivArea",
]

In [40]:
def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = Standalizer(X_scaled)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame(index=X.index)
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new

df.join(cluster_labels(df, cluster_features)).isnull().sum().sum()
cluster_labels(df, cluster_features)

In [41]:
def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = Standalizer(X_scaled)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    # Label features and join to dataset
    X_cd = kmeans.fit_transform(X_scaled)
    X_cd = pd.DataFrame(
        X_cd, 
        columns = [f"Centroid_{i}" for i in range(X_cd.shape[1])],
        index = X.index
    )
    return X_cd

df.join(cluster_distance(df, cluster_features)).isnull().isnull().sum().sum()
cluster_distance(df, cluster_features)

#### PCA
An unsupervised model we used for feature creation. It could be used to decompose the variational structure in the data. The PCA algorithm gave us loadings which described each component of variation, and also the components which were the transformed datapoints. The loadings can suggest features to create and the components we can use as features directly.

In [42]:
pca_features = [
    "GarageArea",
    "YearRemodAdd",
    "TotalBsmtSF",
    "GrLivArea",
]

In [43]:
def pca_components(df, features, standardize=True):
    X = df.copy()
    X = X.loc[:, features]
    # Standardize
    if standardize: Standalizer(X)
    # Create principal components
    pca_model = PCA()
    X_pca = pca_model.fit_transform(X)
    
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(
        X_pca, 
        columns = component_names,
        index = X.index
    )
    
    # Create loadings
    loadings = pd.DataFrame(
        pca_model.components_.T,  # transpose the matrix of loadings
        columns = component_names,  # so the columns are the principal components
        index = X.columns,  # and the rows are the original features
    )
    
    return pca_model, X_pca, loadings

pca_model, X_pca, loadings = pca_components(df, pca_features)
X_pca

In [44]:
def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    fig.set(figwidth=8, dpi=100)
    
    n = pca.n_components_
    print(f"pca n components: {pca.n_components_}")
    grid = np.arange(1, n + 1)
    
    # Explained variance
    evr = pca.explained_variance_ratio_
    print(f"pca  explained variance ratio: {pca.explained_variance_ratio_}")
    
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    #return axs

plot_variance(pca_model, width=8, dpi=100)

## some ideas for other transforms you could explore:


#### Interactions between the quality Qual and condition Cond features. 
    OverallQual, for instance, was a high-scoring feature. 
    You could try combining it with OverallCond by converting both to integer type 
    and taking a product.

#### Square roots of area features. This would convert units of square feet to just feet.

#### Interactions between numeric and categorical features 
that describe the same thing. 
You could look at interactions between BsmtQual and TotalBsmtSF, for instance.

#### Other group statistics in Neighboorhood.
We did the median of GrLivArea. Looking at mean,
std, or count could be interesting. You could also try combining the group statistics with
other features. Maybe the difference of GrLivArea and the median is important?

# Quantile binning

    The only 2-quantile is called the median
    The 3-quantiles are called tertiles or terciles → T
    The 4-quantiles are called quartiles → Q; the difference between upper and lower quartiles is also called the interquartile range, midspread or middle fifty → IQR = Q3 −  Q1
    The 5-quantiles are called quintiles → QU
    The 6-quantiles are called sextiles → S
    The 7-quantiles are called septiles
    The 8-quantiles are called octiles
    The 10-quantiles are called deciles → D
    The 12-quantiles are called duo-deciles or dodeciles
    The 16-quantiles are called hexadeciles → H
    The 20-quantiles are called ventiles, vigintiles, or demi-deciles → V
    The 100-quantiles are called percentiles → P
    The 1000-quantiles have been called permilles or milliles, but these are rare and largely obsolete

## Data Preprocessing Pipline

Before we can do any feature engineering, we need to preprocess the data to get it in a form suitable for analysis. we'll need to:

    Load: the data from CSV files
    Clean: the data to fix any errors or inconsistencies
    Encode: the statistical data type (numeric, categorical)
    Impute: any missing values
    
 After reading the CSV file, we'll apply three preprocessing steps, clean, encode, and impute, and then create the data splits: one (df_train) for training the model, and one (df_test) for making the predictions that you'll submit to the competition for scoring on the leaderboard.

In [45]:
def Data():
    # Read the data
    df_train = pd.read_csv("../input/train.csv", index_col=0)
    df_test = pd.read_csv("../input/test.csv", index_col=0)    
    #df_train = only_once_values(df_train) # apply this in traing dataset ONLY
    
    # delete_uninformative
    df_train = delete_uninformative(df_train)
    print(f"concatenate Done: {df_train.shape}")
    
    target = df_train.pop("SalePrice")
    
    df = pd.concat([df_train, df_test])
    print(f"concatenate Done: {df.shape}")

    # Preprocessing
    df = Clean(df)
    print(f"Clean Done: {df.shape}")
    
    df = Impute(df)
    print(f"Impute Done: {df.shape}")
    
    df = Encoder(df, max_number=5) 
    print(f"Encoder Done: {df.shape}")
    
    # split data
    df_train = df.loc[df_train.index, :]
    df_train['SalePrice'] = target
    df_test = df.loc[df_test.index, :]
    print(df_train.shape, df_test.shape)
    
    return df_train, df_test

#df_train, df_test = tqdm(Data())
#print('Null Values: ', df_train.isnull().sum().sum(), df_test.isnull().sum().sum())

## Create Features Pipline
Now we'll start developing our feature set.

To make our feature engineering workflow more modular, we'll define a function that will take a prepared dataframe and pass it through a pipeline of transformations to get the final feature set.

In [71]:
def create_features(df, df_test):
    X = df.copy()
    target = X.pop("SalePrice")
    
    mi_scores = make_mi_scores(X, target)
    
    # Combine splits if test data is given
    #
    # If we're creating features for test set predictions, we should
    # use all the data we have available. After creating our features,
    # we'll recreate the splits.
    X_test = df_test.copy()
    X = pd.concat([X, X_test])
    print(f"concatenate Done: {X.shape}")
    X_new = pd.DataFrame({}, index = X.index)
    
    #  Mutual Information
    X = drop_uninformative(X, mi_scores, 0.0)
    print(f"drop_uninformative Done: {X.shape}")
    print('Null Values: ', X.isnull().sum().sum())

    # Transformations
    X_new = X_new.join(mathematical_transforms(X))
    print(f"mathematical_transforms Done: {X.shape}")
    
    #X = X.join(interactions(X))
    #print(f"interactions Done: {X.shape}")
    
    X_new = X_new.join(counts(X))
    print(f"counts Done: {X.shape}")
    
    #X_new = X_new.join(Building_Up_Breaking_Down(X))
    #print(f"Building_Up_Breaking_Down Done: {X.shape}")

    X_new = X_new.join(group_transforms(X))
    print(f"group_transforms Done: {X.shape}")
    
    # Clustering
    X_new = X_new.join(cluster_labels(X, cluster_features, n_clusters=20))
    print(f"cluster_labels Done: {X.shape}")

    X_new = X_new.join(cluster_distance(X, cluster_features, n_clusters=20))
    print(f"cluster_distance Done: {X.shape}")

    # PCA
    X_new = X_new.join(pca_components(X, pca_features)[1])
    print(f"pca_components Done: {X.shape}")

    #X_new = X_new.join(indicate_outliers(X))
    
    X = X.join(X_new)
    
    X = Encoder(X, 15)
    print(f"Encoder Done: {X.shape}")
        
    # Scale
    #X = pd.DataFrame(MinMaxScaler().fit_transform(X), index=X.index, columns=X.columns)
    #print(f"scaler Done: {X.shape}")
    
    # ----- Reform splits -----
    X_test = X.loc[df_test.index, :]
    X.drop(X_test.index, inplace=True)
    print(f"drop_uninformative 2 Done: {X.shape}")
    
    X = remove_outliers(X, standard_deviations=1)
    target = target[X.index]
    
    mi_scores = make_mi_scores(X, target)
    X = drop_uninformative(X, mi_scores, 0.0)
    X_test = X_test[X.columns]
    
    return X, target, X_test

df_train, df_test = tqdm(Data())
print('Final shapes: ', df_train.shape, df_test.shape)
print('Null Values: ', df_train.isnull().sum().sum(), df_test.isnull().sum().sum())

features, target, df_test = tqdm(create_features(df_train, df_test))
print('Final shapes: ', features.shape, df_test.shape)
print('Null Values: ', features.isnull().sum().sum(), df_test.isnull().sum().sum())

## Establish Baseline

Finally, let's establish a baseline score to judge our feature engineering against.

This function will compute the cross-validated RMSLE score for a feature set. We've used XGBoost for our model, but you might want to experiment with other models.

## Divide train dataset to train/validate

In [72]:
df_train, df_test = tqdm(Data())
features, target, df_test = tqdm(create_features(df_train, df_test))

train_features, validate_features, train_target, validate_target = train_test_split(
    features, 
    target, 
    test_size=0.25, 
    random_state=42
)

## Hyperparameter Tuning

we do here some hyperparameter tuning with GradientBoostingRegressor.

In [73]:
param_grid = {
    #'fit_intercept':[True,False],
    'n_estimators': [100, 200, 300, 400, 500], 
    'learning_rate': [0.01, 0.05, 0.09, 0.1, 0.2, 0.9],
}
model_param_mod = GridSearchCV(
    estimator = GradientBoostingRegressor(),
    param_grid = param_grid, 
    n_jobs = -1
)

#model_param_mod.fit(train_features, train_target)
#print(model_param_mod.best_params_)

In [74]:
def basic_model(X1, y1, X2, y2):
    # Define model
    #model = XGBRegressor(n_estimators=200, learning_rate=0.09)
    #model = RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt',n_estimators=200)
    #model = BaggingRegressor()
    #model = BayesianRidge()
    #model = Lasso(alpha=100)
    #model = ExtraTreesRegressor(n_estimators=200)
    model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05)
    
    model.fit(X1, y1)
    pred = model.predict(X2)
    print(f"MAE: {mean_absolute_error(y2, pred):.2f}")
    return model

model = basic_model(train_features, train_target, validate_features, validate_target)

In [75]:
validate_target_predictions = model.predict(validate_features).flatten()
print('MAE:', mean_absolute_error(validate_target, validate_target_predictions))
error = validate_target, validate_target_predictions

#### regresion between true and predicted prices

In [76]:
plt.figure(figsize=(14, 10))
plt.scatter(validate_target, validate_target_predictions)
plt.xlabel('True Values [SalePrice]')
plt.ylabel('Predictions [SalePrice]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
## Perfect predictions
plt.plot(validate_target, validate_target,'r');

#### It looks like our model predicts reasonably well. Let’s take a look at the error distribution.

In [77]:
plt.figure(figsize=(14, 12))
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [SalePrice]")
_ = plt.ylabel("Count")

In [78]:
def Save_Submit(model, df):
    output = pd.DataFrame({
        'Id': df.index,
        'SalePrice': model.predict(df).flatten()
    })
    output.to_csv('submission.csv', index=False)
    print("Saved!")
    
model = GradientBoostingRegressor(learning_rate=0.05, n_estimators=300).fit(features, target)
Save_Submit(model, df_test)